In [1]:
import numpy as np
import sys
np.set_printoptions(precision=10, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', True)

In [2]:
# options for mgf
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "6-311G**", ### Split-Valence Basis Set, six Gaussian functions for the core and valence electrons of each atom.
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}


psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)
e, wfn = psi4.energy("scf", return_wfn=True)


In [3]:
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "6-311G**",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.01]),
    'ci_level' : 'cas',
    'nact_orbs' : 8, 
    'nact_els' : 4,
    'full_diagonalization' : True,
    'number_of_photons' : 1
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)

test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)

# test to make sure the CASCI energies match the expected values
expected = np.array([-7.9848177366, -7.8640432702, -7.8443027321, -7.8351500625, -7.7967681454,
            -7.7967681454, -7.790848489,  -7.790848489,  -7.7563223937, -7.7437841386])

#assert np.allclose(test_pf.CIeigs[:10], expected)


Start SCF iterations:

Canonical RHF One-electron energy = -12.5925193790543304
CQED-RHF One-electron energy      = -12.5918498866481059
Nuclear repulsion energy          = 1.0583544213400000
Dipole energy                     = 0.0006804157725302
SCF Iteration   1: Energy = -7.9840375285780469   dE = -7.98404E+00   dRMS = 7.58437E-06
SCF Iteration   2: Energy = -7.9840376228143404   dE = -9.42363E-08   dRMS = 2.66874E-06
SCF Iteration   3: Energy = -7.9840376396926542   dE = -1.68783E-08   dRMS = 7.93097E-07
SCF Iteration   4: Energy = -7.9840376413537752   dE = -1.66112E-09   dRMS = 1.41283E-07
SCF Iteration   5: Energy = -7.9840376413837442   dE = -2.99689E-11   dRMS = 2.29939E-08
SCF Iteration   6: Energy = -7.9840376413847407   dE = -9.96536E-13   dRMS = 1.12146E-09
SCF Iteration   7: Energy = -7.9840376413847371   dE =  3.55271E-15   dRMS = 9.73520E-11
Total time for SCF iterations: 0.010 seconds 

QED-RHF   energy: -7.98403764 hartree
Psi4  SCF energy: -7.98420580 hartree
 Compl

In [4]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50     ## Bondlength
N_el = 50
all_states = np.linspace(0, N_el-1, N_el, dtype=int)       ## linespace generates evenly spaced numbers within a specified interval.
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))
Mu_array = np.zeros((N_R, N_el, N_el, 3))      #######?? Dimensions
psi4.set_options(options_dict)

r_idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    print(mol_str)
    mol = psi4.geometry(mol_str)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)  #### this line performs a self-consistent field (SCF) calculation using Psi4 and returns the SCF energy and the wavefunction object. 
    #fci_energy = psi4.energy('fci',ref_wfn=wfn)    ### FCI calculation is generally feasible only for small systems due to the exponential scaling of the calculation with the number of electrons and basis functions.
                                                    ### Basis set is large. 
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    #assert np.isclose(fci_energy, LiH.CIeigs[0], 1e-7)
    #sort out states with dipole-allowed transition from ground state
    singlet_states = LiH.sort_dipole_allowed_states( N_el)
    print(singlet_states)
    
    E_array[r_idx, :] = LiH.CIeigs[singlet_states]
    Mu_array[r_idx, :, :, :] = LiH.compute_dipole_moments(singlet_states)
    
    # uncomment to try not sorting singlet states
    #E_array[r_idx, :] = LiH.CIeigs[all_states]
    #Mu_array[r_idx, :, :, :] = LiH.compute_dipole_moments(all_states)
    
    r_idx += 1




Li
H 1 1.4
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.7385117713869356
CQED-RHF One-electron energy      = -12.7379011492056993
Nuclear repulsion energy          = 1.1339511657214287
Dipole energy                     = 0.0006176904138339
SCF Iteration   1: Energy = -7.9789411412836326   dE = -7.97894E+00   dRMS = 7.11725E-06
SCF Iteration   2: Energy = -7.9789412209360666   dE = -7.96524E-08   dRMS = 2.45118E-06
SCF Iteration   3: Energy = -7.9789412345063084   dE = -1.35702E-08   dRMS = 7.24961E-07
SCF Iteration   4: Energy = -7.9789412358629690   dE = -1.35666E-09   dRMS = 1.37621E-07
SCF Iteration   5: Energy = -7.9789412358920604   dE = -2.90914E-11   dRMS = 2.24720E-08
SCF Iteration   6: Energy = -7.9789412358930072   dE = -9.46798E-13   dRMS = 1.11243E-09
SCF Iteration   7: Energy = -7.9789412358930072   dE =  0.00000E+00   dRMS = 9.92675E-11
Total time for SCF iterations: 0.010 seconds 

QED-RHF   energy: -7.97894124 hartree
Psi4  SCF energy: -

Generating all determinants in active space
 Completed determinant list in 0.006356000900268555 seconds 
 Completed constant offset matrix in 0.0022652149200439453 seconds
 Completed Hamiltonian build in 6.256285905838013 seconds
[0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 31, 34, 35, 36, 37, 39, 42, 44, 46, 47, 52, 54, 56, 57, 58, 60, 64, 66, 67, 68, 70, 71, 73, 76, 77, 80, 81, 82, 83, 88, 90, 91, 92, 93, 95]

Li
H 1 1.4816326530612245
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.6184915955316725
CQED-RHF One-electron energy      = -12.6178332721168260
Nuclear repulsion energy          = 1.0714745174723141
Dipole energy                     = 0.0006685019814002
SCF Iteration   1: Energy = -7.9834012167822515   dE = -7.98340E+00   dRMS = 7.49759E-06
SCF Iteration   2: Energy = -7.9834013081501407   dE = -9.13679E-08   dRMS = 2.62781E-06
SCF Iteration   3: Energy = -7.9834013243675024   dE = -1.62174E-08   dRMS = 7.80106E-07
SCF Iteration   4: E

Generating all determinants in active space
 Completed determinant list in 0.006365776062011719 seconds 
 Completed constant offset matrix in 0.0021152496337890625 seconds
 Completed Hamiltonian build in 6.24226188659668 seconds
[0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 31, 34, 35, 36, 37, 39, 41, 44, 46, 47, 52, 53, 56, 57, 59, 61, 65, 66, 67, 68, 70, 71, 72, 75, 78, 80, 81, 82, 84, 89, 90, 91, 92, 93, 95]

Li
H 1 1.5632653061224488
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.5058307240581712
CQED-RHF One-electron energy      = -12.5051215230431190
Nuclear repulsion energy          = 1.0155228455416452
Dipole energy                     = 0.0007228262003303
SCF Iteration   1: Energy = -7.9853688110557828   dE = -7.98537E+00   dRMS = 7.88793E-06
SCF Iteration   2: Energy = -7.9853689159071317   dE = -1.04851E-07   dRMS = 2.81355E-06
SCF Iteration   3: Energy = -7.9853689352696273   dE = -1.93625E-08   dRMS = 8.39632E-07
SCF Iteration   4: En

Generating all determinants in active space
 Completed determinant list in 0.006288290023803711 seconds 
 Completed constant offset matrix in 0.0020017623901367188 seconds
 Completed Hamiltonian build in 6.193424224853516 seconds
[0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 31, 34, 35, 36, 37, 39, 40, 44, 46, 47, 52, 53, 57, 58, 60, 61, 65, 66, 67, 68, 70, 71, 72, 75, 78, 80, 81, 82, 85, 89, 90, 91, 92, 93, 95]

Li
H 1 1.6448979591836734
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.4000222937644846
CQED-RHF One-electron energy      = -12.3992589988657933
Nuclear repulsion energy          = 0.9651246894353599
Dipole energy                     = 0.0007807405944476
SCF Iteration   1: Energy = -7.9854396757519090   dE = -7.98544E+00   dRMS = 8.29267E-06
SCF Iteration   2: Energy = -7.9854397961842327   dE = -1.20432E-07   dRMS = 3.01043E-06
SCF Iteration   3: Energy = -7.9854398192867322   dE = -2.31025E-08   dRMS = 9.04079E-07
SCF Iteration   4: E

Generating all determinants in active space
 Completed determinant list in 0.006267070770263672 seconds 
 Completed constant offset matrix in 0.002009153366088867 seconds
 Completed Hamiltonian build in 6.221881866455078 seconds
[0, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 31, 34, 35, 36, 37, 39, 40, 44, 45, 48, 52, 53, 58, 59, 61, 62, 65, 66, 67, 68, 70, 71, 72, 75, 78, 80, 81, 82, 88, 89, 90, 91, 92, 93, 95]

Li
H 1 1.726530612244898
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.3005781116349233
CQED-RHF One-electron energy      = -12.2997574751011349
Nuclear repulsion energy          = 0.9194923164124112
Dipole energy                     = 0.0008423107098191
SCF Iteration   1: Energy = -7.9840727592467804   dE = -7.98407E+00   dRMS = 8.71533E-06
SCF Iteration   2: Energy = -7.9840728977215765   dE = -1.38475E-07   dRMS = 3.22030E-06
SCF Iteration   3: Energy = -7.9840729252664904   dE = -2.75449E-08   dRMS = 9.73978E-07
SCF Iteration   4: Ene

...

Generating all determinants in active space
 Completed determinant list in 0.006328105926513672 seconds 
 Completed constant offset matrix in 0.002040863037109375 seconds
 Completed Hamiltonian build in 6.369222164154053 seconds
[0, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 32, 33, 35, 36, 37, 39, 40, 44, 45, 48, 51, 55, 59, 60, 62, 64, 65, 66, 67, 68, 70, 71, 72, 75, 78, 80, 81, 82, 88, 89, 90, 91, 92, 93, 95]

Li
H 1 1.889795918367347
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.1189939947204195
CQED-RHF One-electron energy      = -12.1180488452700530
Nuclear repulsion energy          = 0.8400545352968682
Dipole energy                     = 0.0009766113463749
SCF Iteration   1: Energy = -7.9783638436110458   dE = -7.97836E+00   dRMS = 9.62512E-06
SCF Iteration   2: Energy = -7.9783640272332779   dE = -1.83622E-07   dRMS = 3.68544E-06
SCF Iteration   3: Energy = -7.9783640662593527   dE = -3.90261E-08   dRMS = 1.13257E-06
SCF Iteration   4: Ene

Generating all determinants in active space
 Completed determinant list in 0.006332874298095703 seconds 
 Completed constant offset matrix in 0.002190828323364258 seconds
 Completed Hamiltonian build in 6.309275150299072 seconds
[0, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 32, 33, 34, 36, 37, 39, 40, 44, 45, 49, 51, 57, 59, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 75, 78, 80, 81, 82, 88, 89, 90, 91, 92, 93, 95]

Li
H 1 1.9714285714285715
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -12.0360452726277423
CQED-RHF One-electron energy      = -12.0350329232721744
Nuclear repulsion energy          = 0.8052696684108697
Dipole energy                     = 0.0010494052414521
SCF Iteration   1: Energy = -7.9745095543325997   dE = -7.97451E+00   dRMS = 1.01168E-05
SCF Iteration   2: Energy = -7.9745097660438740   dE = -2.11711E-07   dRMS = 3.94346E-06
SCF Iteration   3: Energy = -7.9745098123854730   dE = -4.63416E-08   dRMS = 1.22279E-06
SCF Iteration   4: En

Generating all determinants in active space
 Completed determinant list in 0.006415843963623047 seconds 
 Completed constant offset matrix in 0.002535104751586914 seconds
 Completed Hamiltonian build in 6.220778942108154 seconds
[0, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 32, 33, 34, 36, 37, 39, 40, 44, 45, 49, 51, 57, 58, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 75, 78, 80, 81, 82, 88, 89, 90, 91, 92, 93, 95]

Li
H 1 2.053061224489796
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -11.9578391618455235
CQED-RHF One-electron energy      = -11.9567563059515969
Nuclear repulsion energy          = 0.7732509937225647
Dipole energy                     = 0.0011259810950799
SCF Iteration   1: Energy = -7.9702255973559790   dE = -7.97023E+00   dRMS = 1.06361E-05
SCF Iteration   2: Energy = -7.9702258416072898   dE = -2.44251E-07   dRMS = 4.22011E-06
SCF Iteration   3: Energy = -7.9702258965210788   dE = -5.49138E-08   dRMS = 1.32158E-06
SCF Iteration   4: Ene

Generating all determinants in active space
 Completed determinant list in 0.00650787353515625 seconds 
 Completed constant offset matrix in 0.0017619132995605469 seconds
 Completed Hamiltonian build in 6.249876022338867 seconds
[0, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 26, 32, 33, 34, 36, 37, 39, 40, 42, 45, 49, 51, 57, 58, 60, 63, 64, 65, 66, 67, 68, 70, 71, 73, 75, 77, 80, 81, 83, 88, 89, 90, 91, 92, 93, 95]

Li
H 1 2.1346938775510207
symmetry c1


Start SCF iterations:

Canonical RHF One-electron energy = -11.8840471817533846
CQED-RHF One-electron energy      = -11.8828905108212428
Nuclear repulsion energy          = 0.7436811660467495
Dipole energy                     = 0.0012063350019188
SCF Iteration   1: Energy = -7.9656412985429599   dE = -7.96564E+00   dRMS = 1.11851E-05
SCF Iteration   2: Energy = -7.9656415804689962   dE = -2.81926E-07   dRMS = 4.51654E-06
SCF Iteration   3: Energy = -7.9656416453828589   dE = -6.49139E-08   dRMS = 1.43008E-06
SCF Iteration   4: En

Generating all determinants in active space
 Completed determinant list in 0.006310939788818359 seconds 
 Completed constant offset matrix in 0.0021250247955322266 seconds
 Completed Hamiltonian build in 6.2499167919158936 seconds
[0, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 26, 32, 33, 34, 36, 37, 39, 40, 42, 45, 49, 51, 57, 59, 60, 63, 64, 65, 66, 67, 68, 70, 71, 74, 75, 77, 80, 81, 83, 88, 89, 90, 91, 92, 93, 95]


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
print(r_array)
print(E_array[:,1]-E_array[:,0])

#plt.plot(r_array, E_array[:,1]-E_array[:,0])
#plt.plot(r_array, E_array[:,0]+0.1208, label="E0")
plt.plot(r_array, E_array[:,0], label="E0")
plt.plot(r_array, E_array[:,1], 'ro', label="E1")
plt.plot(r_array, E_array[:,2], 'bo', label="E2")
plt.plot(r_array, E_array[:,4], label="E3")
plt.legend()
# plt.xlim(1.79, 1.81)
plt.show()

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
def build_pf_hamiltonian(n_el, n_ph, E_R, omega, lamvec, mu):
    """
    Given an array of n_el E_R values and an n_ph states with fundamental energy omega
    build the PF Hamiltonian
    
    n_el : int
        the number of electronic states (n_el = 1 means only ground-state)
    
    n_ph : int
        the number of photon occupation states (n_ph = 1 means only the |0> state)
    
    E_R : np.array of floats
        the electronic energies
        
    omega : float
        the energy of the photonic mode
        
    lamvec : np.array of floats
        the lambda vector
        
    mu : (n_el x n_el x 3) np.array of floats 
        mu[i, j, k] is the kth cartesian component of the dipole moment expectation value between 
        state i and state j
    
    """
    H_PF = np.zeros((n_el * n_ph, n_el * n_ph))
    
    # take care of the diagonals first
    # bare electronic and photonic energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            H_PF[na,na] = E_R[a] + n * omega
        
    # diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for g in range(n_el):
                H_PF[na,na] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g,a,:])
            
    # off-diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for b in range(n_el):
                nb = n * n_el + b
                for g in range(n_el):
                    if a != b:
                        H_PF[na, nb] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g, b, :])
                
    # off-diagonal bilinear coupling
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            
            for m in range(n_ph):
                for b in range(n_el):
                    mb = m * n_el + b
                    
                    if n == (m-1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na,mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        
                    elif n == (m+1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na, mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1)
                        
    
    return H_PF
                
            
    
        
    
    

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
np.set_printoptions(precision=6, linewidth=200, suppress=True)
omega_cav = 0.12086
lambda_vector = np.array([0., 0., 0.08])      ##### This is the value you have to change and try..........
N_el = 3
N_ph = 2
pcqed_nel3_np2 = np.zeros((N_R, N_el * N_ph ))
_HPF = build_pf_hamiltonian(N_el, N_ph, E_array[0,:], omega_cav, lambda_vector, Mu_array[0, :, :, :])
print(_HPF)

We want to investigate the mean absolute error of a given polariton potential energy surface as a 
function of electronic and photonic truncation:

$$ {\rm MAE} = \frac{1}{N_R} \sum_i^{N_R} \left| E(R_i) - E^{'}(R_i) \right| $$

where $E^{'}(R_i)$ is a value of the polaritonic PES for a truncated level of theory and $E(R_i)$ is the exact value.

In [ ]:
def MAE(E, Ep):
    """
    function to return the mean absolute error of the approximate PES stored in Ep relative to 
    the exact PES stored in E
    """
    # get length
    _NR = len(E)
    
    # take element-by-element difference
    _diff = E - Ep
    
    # compute absolute value of differences
    _absdiff = np.abs(_diff)
    
    # return sum divided by _NR
    return np.sum(_absdiff) / _NR

    

In [ ]:
from matplotlib import cm
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 12
from matplotlib import pyplot as plt

In [ ]:
N_el = 30
N_ph = 20
pcqed_nel30_np20 = np.zeros((N_R, N_el * N_ph ))
for k in range(N_R):           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)   
    pcqed_nel30_np20[k,:] = pf_e

In [ ]:
N_el = 10
N_ph = 2
pcqed_nel10_np2 = np.zeros((N_R, N_el * N_ph ))
for k in range(N_R):           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)    
    pcqed_nel10_np2[k,:] = pf_e

In [ ]:
N_el = 50
N_ph = 7
pcqed_nel50_np7 = np.zeros((N_R, N_el * N_ph ))
for k in range(N_R):           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)    
    pcqed_nel50_np7[k,:] = pf_e

In [ ]:
# N_el = 50
# N_ph = 50
# pcqed_nel50_np50 = np.zeros((N_R, N_el * N_ph ))
# for k in range(N_R):           
#     _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
#     pf_e, pf_c = np.linalg.eigh(_HPF)    
#     pcqed_nel50_np50[k,:] = pf_e

In [ ]:
N_el = 5
N_ph = 5
pcqed_nel5_np5 = np.zeros((N_R, N_el * N_ph ))
for k in range(N_R):           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)    
    pcqed_nel5_np5[k,:] = pf_e

In [ ]:
plt.plot(r_array, E_array[:,0]+0.1208, label="E0")
plt.plot(r_array, E_array[:,0], 'black', linestyle="dashed", label="No Cavity")
plt.plot(r_array, pcqed_nel30_np20[:,0], 'red', label="pCQED (30,20)")
plt.plot(r_array, pcqed_nel10_np2[:,0], 'bo', label="pCQED (10,2)")
plt.plot(r_array, pcqed_nel50_np7[:,0], 'purple', linestyle="dashed", label="pCQED (50,7)")
# plt.plot(r_array, pcqed_nel50_np50[:,0], 'yo', label="pCQED (50,50)")
plt.plot(r_array, pcqed_nel5_np5[:,0], 'yellow',label="pCQED (5,5)")
# plt.xlim(1.4, 2.0)
# plt.ylim(-7.885, -7.855)
plt.ylabel("Energy (Hartree)")
plt.xlabel("Bondlength (Angstroms)")
plt.legend()
plt.show()

In [ ]:
plt.plot(r_array, E_array[:,0]+0.12086, 'grey', linestyle="dashed", label="Uncoupled")
plt.plot(r_array, E_array[:,1], 'grey', linestyle="dashed")#, label="Ee Uncoupled")
plt.plot(r_array, pcqed_nel10_np2[:,1], 'red', label="pCQED (10,2)")
plt.plot(r_array, pcqed_nel10_np2[:,2], 'red')# label="pCQED (10,2)")
plt.plot(r_array, pcqed_nel50_np7[:,1], 'blue',  label="pCQED (50,7)")
plt.plot(r_array, pcqed_nel50_np7[:,2], 'blue')#,  label="pCQED (50,7)")
plt.plot(r_array, pcqed_nel30_np20[:,1], 'ro', ms = 3, mfc = 'none', label="pcqed (30,20)")
plt.plot(r_array, pcqed_nel30_np20[:,2], 'ro', ms = 3, mfc = 'none')
# plt.plot(r_array, pcqed_nel50_np50[:,1], 'yo', ms = 3, mfc = 'none', label="pcqed (50,50)")
# plt.plot(r_array, pcqed_nel50_np50[:,2], 'yo', ms = 3, mfc = 'none')
plt.plot(r_array, pcqed_nel5_np5[:,1], 'yellow', ms = 3, mfc = 'none', label="pcqed (5,5)")
plt.plot(r_array, pcqed_nel5_np5[:,2], 'yellow', ms = 3, mfc = 'none')
# plt.xlim(1.4, 2.0)
# plt.ylim(-7.765, -7.725)
plt.ylabel("Energy (Hartree)")
plt.xlabel("Bondlength (Angstroms)")
plt.legend()
plt.savefig("sto3g_lih_lp_up_lambda_05.png")
plt.show()

In [ ]:
## How to add scqed states for comparsion?
## Analysis from different graphs

